# 理解并实现ResNet和ResNext：从Microsoft到Facebook[Part 1]
在本博客中我们将探索残差网络。具体来讲我们将讨论微软研究院和脸书AI研究院发布的三篇论文——ResNet和ResNext，并且尝试用Pytorch来实现。<br>
### 关于这个系列
本系列包含两个部分，分别是：<br>
- 理解和实现ResNet结构
- 理解和实现ResNeXt结构
我们将回顾一下三篇介绍和改进残差网络的论文：
- [PART-1][Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf)
- [PART-2][Identity mappings in Deep Residual Networks](https://arxiv.org/pdf/1603.05027.pdf)
- [PART-3][Aggregated Residual Transformation for Deep Neural Networks](https://arxiv.org/pdf/1611.05431.pdf)
### ResNet有多成功？
- 赢得了ILSVRC 2015分类挑战赛的冠军，top-5的错误率是3.57%（一个集成模型）。
- 赢得了ILSVRC和COCO2015 ImageNet检测和定位以及COCO检测和分割比赛的冠军。
- 将Faster R-CNN中的VGG-16替换为ResNet-101，精度提升了28%。
- 可以对100层甚至1000层的网络进行有效训练。

## ResNet解决了什么问题？
### 问题：
当深度网络开始收敛，一个难题开始显现：随着网络深度增加，精确率会饱和并且迅速下降。<br>
### 分步分析下降原因：
我们先从较浅的网络来看，然后逐步增加网络的深度。<br>
### 最差场景：
更深网络的前层可以用较浅的网络替换，剩余的层可以视为一个恒等函数（输出等于输入）。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*1y9hueMSZAeo1Hbp9KYKiw.png)
### 奖励场景：
在深度网络中附加的层可以更好的近似映射大幅减少误差。<br>
### 实验：
在最差场景下，前层网络和它的深层部分得到的是相同的精度。在奖励场景中，深层模型相较于前层网络能够达到更高的精度。但是我们的实验表明深度模型表现得并不好。所以使用深度网络反而拉低了模型的性能。这篇文章尝试通过使用**深度残差学习框架**来解决这个问题。<br>
### 如何解决？
不同于直接使用一个函数H(x)来映射x->y（一些非线性变换层）。我们将残差函数定义为F(x)=H(x)-x，可以进一步变换为H(x)=F(x)+x，这里F(x)和x表示一系列非线性层和恒等函数（输入等于输出）。<br>

## 作者的假设是优化残差映射函数F(x)相较于原始函数H(x)更加容易。

## 残差学习模块原理
如果恒等映射是最优的，我们可以轻易地讲残差函数优化为零（F(x)=0），相较之下由一系列非线性层的基础上去训练为恒等映射就难的多。简言之就是提出一个类似于F(x)=0的解决方案相较于使用一些列非线性的cnn网络来表示为F(x)=x的解决方案要容易得多。所以，函数F(x)就是作者所谓的残差函数。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*WVs9ywVLLKjSUBZ_mnfFrw.png)
作者做了一些测试来验证他的假设。让我们来看一下。<br>
### 测试案例
有一个平坦网络（如VGG18），称为网络1，以及一个更深层的平坦网络（34层），称为网络2，在网络2的基础上加一个残差层得到网络3。<br>
### 设计网络：
1. 大部分情况使用3*3卷积
2. 降采样使用步长为2
3. 全局平均池化层以及一个维度为1000的全连接层，最后跟Softmax
![avater](https://cdn-images-1.medium.com/max/800/1*4tlPOipWjcwIoNUlQ6IWFQ.png)

有两种类型的残差连接：<br>
![avater](https://cdn-images-1.medium.com/max/800/1*tEaVn-9OEPCre0lmHAJWyw.png)
1. 短路连接在输入输出维度相同时可以直接使用。
![avater](https://cdn-images-1.medium.com/max/800/1*37brTipLpo6naVYHiXMbsg.png)
2. 当维度改变时，有两种情况，一种情况是短路连接仍然使用恒等映射，但是输入上需要在多余的维度上补零。另一种情况是使用下面公式来匹配维度（通过1*1卷积）<br>
![avater](https://cdn-images-1.medium.com/max/800/1*07wrOB82Ktl3uWhY0GWE7A.png)

第一个例子没有引入新的参数，而第二个引入了新的参数W_{s}。
### 结果：
尽管18层网络是34层网络的子集，它的表现却更好。ResNet在深度网络上的表现要好很多。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*-_ED04HNCNz7HFyqQtbTtg.png)

### 深层学习
下面网络是学习得到的<br>
![avater](https://cdn-images-1.medium.com/max/800/1*aq0q7gCvuNUqnMHh4cpnIw.png)
每个残差块可以是两层（在最小的网络如ResNet18和34中使用）或者三层（ResNet50,101,152中使用）。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*zS2ChIMwAqC5DQbL5yD9iQ.png)
实现可以参见[Pytorch实现](https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py)。
该代码中，Bottleneck类实现了一个3层的卷积块，而Basicblock实现了一个两层的卷积。同时还实现了ImageNet上所有预训练的ResNet结构。

### 观察
1. ResNet相较于平整网络收敛得更快。
2. 恒等vs投影短路。在所有层加上投影短路（公式2）获得的改进微乎其微。所以所有的ResNet块仅使用恒等短路，只在需要维度改变的时候应用投影短路。
3. ResNet-34在top-5的验证误差是5.71%，比BN-Inception以及VGG表现的好。ResNet-152的top-5验证集误差是4.99%。6个不同深度的模型集成后达到了3.57%的top5验证集误差。赢得了ILSVRC-2015的冠军。
![avater](https://cdn-images-1.medium.com/max/800/1*m8GPYCU9c02ftwDk8r_8Fg.png)


### Pytorch下的实现
我有几乎所有的图像分类网络[实现](https://medium.com/@14prakash/almost-any-image-classification-problem-using-pytorch-i-am-in-love-with-pytorch-26c7aa979ec4)，这些实现可以让你快速地实现和训练ResNet。Pytorch已经有了自己的实现，我的实现是考虑了迁移学习的不同情况。<br>
我写了一篇详细的博客来介绍[迁移学习](https://medium.com/@14prakash/transfer-learning-using-keras-d804b2e04ef8)。尽管代码是用keras实现的，但是背后的思想是更加抽象的，对于理解原理非常有帮助。